# Setting paths

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from skimage.io import imread
import cv2

from tqdm import tqdm
from skimage.util import montage
from skimage.morphology import label
import logging
import sys

import numpy as np
import torch
import torch.nn as nn
from torch import optim
from tqdm import tqdm

from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import DataLoader, random_split

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

print(os.listdir("../input/almaz-antey-hackathon-l1"))

# Any results you write to the current directory are saved as output.

['train', 'train_segmentation.csv', 'test', 'sample_submission.csv']


In [2]:
! pip install segmentation-models-pytorch

     |████████████████████████████████| 53 kB 529 kB/s eta 0:00:011
     |████████████████████████████████| 58 kB 1.8 MB/s eta 0:00:011
     |████████████████████████████████| 161 kB 3.1 MB/s eta 0:00:01
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.6.3-py3-none-any.whl size=12419 sha256=64c316605f11e1df3d05ecbed52bde1bb5830021434780e0e728102b4e892c9e
  Stored in directory: /root/.cache/pip/wheels/90/6b/0c/f0ad36d00310e65390b0d4c9218ae6250ac579c92540c9097a
  Created wheel for pretrainedmodels: filename=pretrainedmodels-0.7.4-py3-none-any.whl size=60962 sha256=4200974d58fdf3a5cef30e947edc85609633350c6339246fad8b5029dd3af75d
  Stored in directory: /root/.cache/pip/wheels/ed/27/e8/9543d42de2740d3544db96aefef63bda3f2c1761b3334f4873
Successfully built efficientnet-pytorch pretrainedmodels


In [3]:
!pip install albumentations

# Load data

In [4]:
train = os.listdir('../input/almaz-antey-hackathon-l1/train/train')
print(len(train))

test = os.listdir('../input/almaz-antey-hackathon-l1/test/test')
print(len(test))

16343
2957


In [5]:
ship_dir = '../input/almaz-antey-hackathon-l1/'
train_image_dir = os.path.join(ship_dir, 'train/train')
test_image_dir = os.path.join(ship_dir, 'test/test')

In [6]:
# load csv-data files
train_df = pd.read_csv(os.path.join(ship_dir, 'train_segmentation.csv'))
sample_sub = pd.read_csv(os.path.join(ship_dir, 'sample_submission.csv'))

In [7]:
train_df.head()

,Unnamed: 0,ImageId,EncodedPixels
0,12166,000155de5.jpg,264661 17 265429 33 266197 33 266965 33 267733...
1,2341,000194a2d.jpg,51834 9 52602 9 53370 9 54138 9 54906 9 55674 ...
2,1981,00021ddc3.jpg,74441 7 75207 9 75972 12 76738 14 77506 10 775...
3,0,0005d6d95.jpg,265143 1 265910 4 266678 5 267445 7 268212 10 ...
4,5703,0017c19d6.jpg,329228 1 329995 3 330762 4 331529 6 332296 8 3...


# Run Length Decoding

In [64]:
montage_rgb = lambda x: np.stack([montage(x[:, :, :, i]) for i in range(x.shape[3])], -1)

def multi_rle_encode(img, **kwargs):
    '''
    Encode connected regions as separated masks
    '''
    labels = label(img)
    if img.ndim > 2:
        return [rle_encode(np.sum(labels==k, axis=2), **kwargs) for k in np.unique(labels[labels>0])]
    else:
        return [rle_encode(labels==k, **kwargs) for k in np.unique(labels[labels>0])]

# ref: https://www.kaggle.com/paulorzp/run-length-encode-and-decode
def rle_encode(img, min_max_threshold=1e-4, max_mean_threshold=None):
    '''
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    '''
    if np.max(img) < min_max_threshold:
        return '' ## no need to encode if it's all zeros
    if max_mean_threshold and np.mean(img) > max_mean_threshold:
        return '' ## ignore overfilled mask
    pixels = img.T.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

def rle_decode(mask_rle, shape=(768, 768)):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (height,width) of array to return 
    Returns numpy array, 1 - mask, 0 - background
    '''
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape).T  # Needed to align to RLE direction

def masks_as_image(in_mask_list, **kwargs):
    # Take the individual ship masks and create a single mask array for all ships
    all_masks = np.zeros(kwargs['shape'], dtype = np.uint8)
    for mask in in_mask_list:
        if isinstance(mask, str):
            all_masks |= rle_decode(mask, **kwargs)
    return all_masks

def masks_as_color(in_mask_list, **kwargs):
    # Take the individual ship masks and create a color mask array for each ships
    all_masks = np.zeros(kwargs['shape'], dtype = np.float)
    scale = lambda x: (len(in_mask_list) + x + 1) / (len(in_mask_list) * 2) ## scale the heatmap image to shift 
    for i,mask in enumerate(in_mask_list):
        if isinstance(mask, str):
            all_masks[:,:] += scale(i) * rle_decode(mask, **kwargs)
    return all_masks

# Simple data generator

In [9]:
def ship_generator(database, image_path, batch_size=9):
    all_batches = list(database.groupby('ImageId'))
    out_rgb = []
    out_masks = []
    while True:
        np.random.shuffle(all_batches)  # shuffle
        for c_img_id, c_masks in all_batches:
            rgb_path = os.path.join(image_path, c_img_id)
            c_img = imread(rgb_path)
            c_mask = np.expand_dims(masks_as_color(c_masks['EncodedPixels'].values, shape=(768, 768)), -1)
                
            out_rgb += [c_img]
            out_masks += [c_mask]
            if len(out_rgb) >= batch_size:
                yield np.stack(out_rgb, 0)/255.0, np.stack(out_masks, 0)
                out_rgb = []
                out_masks = []

**Torch Dataset**

In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from os.path import splitext
from os import listdir
import numpy as np
from glob import glob
import torch
from torch.utils.data import Dataset
import logging
from PIL import Image

In [11]:
class BasicDataset(Dataset):
    def __init__(self, database, image_path, preprocessing=None):
        self.database = database
        self.image_path = image_path
        self.all_batches = list(database.groupby('ImageId'))
        self.preprocessing = preprocessing

    def __len__(self):
        return len(self.all_batches)
    
    
    def __getitem__(self, idx):
        c_img_id, c_masks = self.all_batches[idx]
        rgb_path = os.path.join(self.image_path, c_img_id)
        c_img = imread(rgb_path)
        c_mask = np.expand_dims(masks_as_color(c_masks['EncodedPixels'].values, shape=(768, 768)), -1)
        
        # apply preprocessing
        if self.preprocessing:
            sample = self.preprocessing(image=c_img, mask=c_mask)
            image, mask = sample['image'], sample['mask']
            
        return image, mask

**Creating Model**

In [12]:
import segmentation_models_pytorch as smp
import albumentations as albu

In [13]:
def to_tensor(x, **kwargs):
    return x.transpose(2, 0, 1).astype('float32')


def get_preprocessing(preprocessing_fn):
    _transform = [
        albu.Lambda(image=preprocessing_fn),
        albu.Lambda(image=to_tensor, mask=to_tensor),
    ]
    return albu.Compose(_transform)

In [14]:
ENCODER = 'se_resnext50_32x4d'
ENCODER_WEIGHTS = 'imagenet'
CLASSES = ['ship']
ACTIVATION = 'sigmoid'
DEVICE = 'cuda'

model = smp.Unet(
    encoder_name=ENCODER, 
    encoder_weights=ENCODER_WEIGHTS, 
    classes=len(CLASSES), 
    activation=ACTIVATION,
)

preprocessing_fn = smp.encoders.get_preprocessing_fn(ENCODER, ENCODER_WEIGHTS)

Downloading: "http://data.lip6.fr/cadene/pretrainedmodels/se_resnext50_32x4d-a260b3a4.pth" to /root/.cache/torch/hub/checkpoints/se_resnext50_32x4d-a260b3a4.pth


In [21]:
val_percent = 0.1

dataset = BasicDataset(train_df, train_image_dir, get_preprocessing(preprocessing_fn))
n_val = int(len(dataset) * val_percent)
n_train = len(dataset) - n_val
train, val = random_split(dataset, [n_train, n_val])
train_loader = DataLoader(train, batch_size=4, shuffle=True, num_workers=12)
valid_loader = DataLoader(val, batch_size=1, shuffle=False, num_workers=4)

In [22]:
loss = smp.utils.losses.DiceLoss()
metrics = [
    smp.utils.metrics.IoU(threshold=0.5),
]

optimizer = torch.optim.Adam([ 
    dict(params=model.parameters(), lr=0.0001),
])

In [24]:
train_epoch = smp.utils.train.TrainEpoch(
    model, 
    loss=loss, 
    metrics=metrics, 
    optimizer=optimizer,
    device=DEVICE,
    verbose=True,
)

valid_epoch = smp.utils.train.ValidEpoch(
    model, 
    loss=loss, 
    metrics=metrics, 
    device=DEVICE,
    verbose=True,
)

In [25]:
max_score = 0

for i in range(0, 5):
    
    print('\nEpoch: {}'.format(i))
    train_logs = train_epoch.run(train_loader)
    valid_logs = valid_epoch.run(valid_loader)
    
    # do something (save model, change lr, etc.)
    if max_score < valid_logs['iou_score']:
        max_score = valid_logs['iou_score']
        torch.save(model, 'best_model.pth')
        print('Model saved!')


Epoch: 0
train:  72%|███████▏  | 2666/3678 [34:40<13:09,  1.28it/s, dice_loss - 0.1817, iou_score - 0.7069]


KeyboardInterrupt: 

In [31]:
best_model = torch.load('first_model_1.pth')

In [32]:
test_epoch = smp.utils.train.ValidEpoch(
    best_model, 
    loss=loss, 
    metrics=metrics, 
    device=DEVICE,
    verbose=True,
)

In [33]:
logs = test_epoch.run(valid_loader)

valid: 100%|██████████| 1634/1634 [01:59<00:00, 13.70it/s, dice_loss - 0.3283, iou_score - 0.577] 


# Create final submission

In [34]:
def get_test_preprocessing(preprocessing_fn):
    _transform = [
        albu.Lambda(image=preprocessing_fn),
        albu.Lambda(image=to_tensor),
    ]
    return albu.Compose(_transform)

In [65]:
pbar = tqdm(sample_sub.index[:])
best_model.eval()
for idx in pbar:  
    fpath = os.path.join(test_image_dir, sample_sub.iloc[idx].ImageId)
    c_img = imread(fpath)
    processed_image = get_test_preprocessing(preprocessing_fn)(image=c_img)['image']
    x_tensor = torch.from_numpy(processed_image).to(DEVICE).unsqueeze(0)
    
    mask = best_model.predict(x_tensor)
    mask_ready = mask.cpu().squeeze(0).numpy()[0] > 0.5
    encode_mask = rle_encode(mask_ready)
    sample_sub.iloc[idx].EncodedPixels = encode_mask

100%|██████████| 2829/2829 [04:27<00:00, 10.57it/s]


In [66]:
sample_sub.to_csv('submission.csv', index=False)
sample_sub.head()

,ImageId,EncodedPixels
0,0010551d9.jpg,183033 13 183799 22 184566 26 185333 29 186100...
1,002a943bf.jpg,403196 3 403959 9 404724 12 405490 15 406256 1...
2,0035268d9.jpg,455522 1 456290 2 457057 3 457825 3 458593 3 4...
3,008f038d3.jpg,225018 5 225777 15 226540 21 227305 24 228067 ...
4,009bc4be5.jpg,161347 5 162112 11 162878 16 163644 19 164411 ...


In [67]:
len(sample_sub)

2829